In [70]:
import webbrowser
import selenium
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller
import os
import numpy as np
import pandas as pd
import re

---  
크롬드라이버 자동설치, headless

In [2]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
driver_path = f'./{chrome_ver}/chromedriver.exe'
if os.path.exists(driver_path):
    print(f"chrom driver is insatlled: {driver_path}")
else:
    print(f"install the chrome driver(ver: {chrome_ver})")
    chromedriver_autoinstaller.install(True)

options = wb.ChromeOptions()
options.add_argument('headless')

chrom driver is insatlled: ./105/chromedriver.exe


---  
크롤링 클릭 함수

In [3]:
# XPATH 클릭 함수
def click(x) :
    driver.find_element(By.XPATH,  value=x).click()
    time.sleep(1)

# 달력 클릭 함수
def callendar() :
    click(start_calander) # 관측시작 달력 클릭
    click(year_box) # 년 텍스트박스 클릭 
    click(select_year) # 관측시작할 년 클릭
    click(month_box) # 월 텍스트박스 클릭
    click(select_mon) # 관측시작할 월 클릭
    click(start_date) # 관측시작할 일 클릭
    
    click(end_calander) # 관측종료 달력 클릭
    click(year_box) # 년 텍스트박스 클릭
    click(select_year2) # 관측시작할 년 클릭
    click(month_box) # 월 텍스트박스 클릭
    click(select_mon) # 관측시작할 월 클릭
    click(end_date) # 관측종료할 일 클릭
    click(search)

---  
크롤링

In [129]:
year = int(input("확인할 년도 입력"))
mon = int(input("확인할 월 입력"))

# XPATH 클릭 함수
def click(x) :
    driver.find_element(By.XPATH,  value=x).click()
    time.sleep(1)
    
# 달력 클릭 함수
def callendar() :
    click(start_calander) # 관측시작 달력 클릭
    click(year_box) # 년 텍스트박스 클릭
    
    # click(select_year) # 관측시작할 년 클릭
    click(select_year)
    
    click(month_box) # 월 텍스트박스 클릭
    click(select_mon) # 관측시작할 월 클릭
    click(start_date) # 관측시작할 일 클릭
    
    click(end_calander) # 관측종료 달력 클릭
    click(year_box) # 년 텍스트박스 클릭
    
    # click(select_year) # 관측시작할 년 클릭
    click(select_year2)
    
    click(month_box) # 월 텍스트박스 클릭
    click(select_mon) # 관측시작할 월 클릭
    click(end_date) # 관측종료할 일 클릭
    click(search)

url = 'https://www.meis.go.kr/mei/observe/wemosensor.do#'
driver = wb.Chrome(options=options)
driver.get(url)

driver.implicitly_wait(5)

# 센서측정자료 클릭하기
x1 = "//button[text()='해양수질자동측정망(센서자료)']"
click(x1)

x2 = '//*[@id="stnpnt_code"]'
click(x2)

# 위치 지정
loc_list = ['시화조력','새만금']
loc = f"//option[text()='{loc_list[0]}']"
click(loc)


# 관측기간 설정하기
date = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
start_calander = '//*[@id="searchForm"]/div/div[2]/span[1]/button'
year_box = '//*[@id="ui-datepicker-div"]/div/div/select[1]'
month_box = '//*[@id="ui-datepicker-div"]/div/div/select[2]'
end_calander = '//*[@id="searchForm"]/div/div[2]/span[3]/button'
search = '//*[@id="searchForm"]/div/div[3]/button[2]'

select_year = f'//*[@id="ui-datepicker-div"]/div/div/select[1]/option[{year-2001}]'
select_year2 = f'//*[@id="ui-datepicker-div"]/div/div/select[1]/option[1]'

select_mon = f"//option[text()='{mon}월']"

start_date = f"//a[text()='{date[1]}']"
if mon == 2 :
    end_date = f"//a[text()='{date[28]}']"
    callendar()
elif mon == 4 or 6 or 9 or 11 :
    end_date = f"//a[text()='{date[30]}']"
    callendar()
else :
    end_date = f"//a[text()='{date[31]}']"
    callendar()
    
# 각 페이지의 결과를 담을 리스트
list = [] 
# 1페이지의 결과
table = driver.find_element(By.XPATH,'//*[@id="mySheet-table"]')
# 페이지를 넘기기위한 번호
path ='//*[@id="listDiv"]/div[4]/button'
# 총 페이지 수
temp = driver.find_element(By.XPATH,'//*[@id="pageCount"]/div/p')
print(temp.text)
result = re.compile('[0-9]+').findall(temp.text) 
print(result)
cnt = int(result[3])
print(cnt)

# 크롤링해오기
try :
    for replay in range ((cnt//10)+1) :
        if (replay == 0) :
            for j in range(3,13) : # 1-10 페이지
                if (j<12) :
                    list.extend(table.text[45:].split('\n'))
                    list.pop() # 결측치 제거
                    click(path+'['+str(j)+']')
                else :
                    click(path+'['+str(12)+']/i') # 다음페이지
        else :        
            for k in range(4,14) : # 11~ 끝페이지
                if (k<13) :
                    list.extend(table.text[45:].split('\n'))
                    list.pop() # 결측치 제거
                    click(path+'['+str(k)+']')
                else :
                    click(path+'['+str(13)+']/i') # 다음페이지
except :
    pass

driver.close()

총 8,580건 입니다.(1 / 858 페이지)
['8', '580', '1', '858']
858


---  
데이터 전처리

In [112]:
table_list = []
for i in list :
    table_list.append(i.split(' '))
# table_list.pop()

In [114]:
a=[] 
b=[]
c=[]
d=[]
e=[]
f=[]
g=[]
h=[]

In [115]:
for i in range(len(table_list)) :
    a.append(table_list[i][0])
    b.append(table_list[i][1])
    c.append(table_list[i][2])
    d.append(table_list[i][3])
    e.append(table_list[i][4])
    f.append(table_list[i][5])
    g.append(table_list[i][6])
    h.append(table_list[i][7])

In [116]:
dict = { '정점명':a,'관측일시' : b, '전기전도도' : c, '수온' : d, '용존산소' : e, '탁도' : f, '염분' : g }
dict.keys()

dict_keys(['정점명', '관측일시', '전기전도도', '수온', '용존산소', '탁도', '염분'])

In [117]:
df = pd.DataFrame(dict)
df

,정점명,관측일시,전기전도도,수온,용존산소,탁도,염분
0,시화조력,2020-07-30,23:55,42.86,24.71,7.86,6.02
1,시화조력,2020-07-30,23:50,42.87,24.71,7.84,5.59
2,시화조력,2020-07-30,23:45,42.87,24.72,7.86,6.05
3,시화조력,2020-07-30,23:40,42.87,24.72,7.86,6.08
4,시화조력,2020-07-30,23:35,42.88,24.73,7.87,6.16
...,...,...,...,...,...,...,...
175,시화조력,2020-07-30,08:30,42.84,24.73,7.86,6.00
176,시화조력,2020-07-30,08:25,42.84,24.73,7.86,6.02
177,시화조력,2020-07-30,08:20,42.84,24.73,7.86,6.00
178,시화조력,2020-07-30,08:15,42.84,24.73,7.86,5.95


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   정점명     180 non-null    object
 1   관측일시    180 non-null    object
 2   전기전도도   180 non-null    object
 3   수온      180 non-null    object
 4   용존산소    180 non-null    object
 5   탁도      180 non-null    object
 6   염분      180 non-null    object
dtypes: object(7)
memory usage: 10.0+ KB
